In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
print(df_train.columns.values)
print(df_test.columns.values)

#3 more features in the train set.

casual - number of non-registered user rentals initiated

registered - number of registered user rentals initiated

count - number of total rentals


In [ ]:
df_train[:10]

In [ ]:
#Custom Transformer which convert datetime column into hour, day, month, year, dayofweek columns
from sklearn.base import TransformerMixin
class TimeTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        temp=pd.DatetimeIndex(X['datetime'])
        cols = ['hour', 'day', 'month', 'year', 'dayofweek']
        X['hour'] = temp.hour
        X['day'] = temp.day
        X['month'] = temp.month
        X['year'] = temp.year
        X['dayofweek'] = temp.dayofweek
        return X

In [ ]:
#Custom Transformer for dop a column
class DropTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        Xnew = X.drop(['datetime'], axis=1)
        return Xnew

In [ ]:
#Custom Transformer change the category of a column
class CategoryTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        categoryVariableList = ["hour","dayofweek","month","season","weather","holiday","workingday"]
        for var in categoryVariableList:
            X[var] = df_train[var].astype("category")
        return X

In [ ]:
#Pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
        ('time', TimeTransformer()),
        ('drop', DropTransformer()),
        ('category', CategoryTransformer()),   
    ])

df_train_tr = pipeline.fit_transform(df_train)
df_test_tr = pipeline.transform(df_test)

In [ ]:
#Correlation matrix between feetures
corr_matrix=df_train.corr()
corr_matrix['count']

Lot of linear relation : 
Temp & Temp proportional
Hour
Humidity

In [ ]:
sns.heatmap(corr_matrix)

In [ ]:
from math import log, exp

X = df_train.drop(['casual', 'registered', 'count', 'datetime'], axis=1)
Y = df_train['count']
Ylog = df_train["count"].apply(log)


In [ ]:
#On test un autre modèle parce que là c'est pas ouf
#Decition Tree
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X, Y)

#Pas d'erreur
tree_reg_predictions = tree_reg.predict(X)
tree_mse = mean_squared_error(labels, tree_reg_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(Y, tree_reg_predictions)
lin_mae

In [ ]:
df_test_datetime = df_test['datetime']
X2 = df_test.drop(['datetime'], axis=1)
tree_reg_predictions_test = tree_reg.predict(X2)
tree_reg_predictions_test
tree_reg_predictions_test_df = pd.DataFrame(tree_reg_predictions_test)
tree_reg_predictions_test_df['count'] = tree_reg_predictions_test_df
df=pd.DataFrame({'datetime':df_test_datetime, 'count':tree_reg_predictions_test})
df.to_csv('Decision file.csv', sep=',', encoding = 'utf-8', index=False)


In [ ]:
#On fit la random forest
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg_log =RandomForestRegressor(random_state=42)
#forest_reg.fit(X, Y)
forest_reg_log.fit(X, Ylog)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

#Cross validatoin sur le Random Forest
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, X, Y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [20, 30, 40], 'max_features': [6, 8, 10, 12]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [20, 30], 'max_features': [6, 8, 10]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X, Y)
grid_search_log = GridSearchCV(forest_reg_log, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search_log.fit(X,Ylog)

In [ ]:
forest_reg_best = grid_search.best_estimator_
forest_reg_best_log = grid_search_log.best_estimator_

In [ ]:
grid_search.best_params_
grid_search_log.best_params_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
#forest_reg_predictions_test = forest_reg.predict(X2)
forest_reg_predictions_test_log = forest_reg_log.predict(X2)
#forest_reg_prediction_test_best = forest_reg_best.predict(X2)
forest_reg_prediction_test_best_log = forest_reg_best_log.predict(X2)

forest_reg_predictions_test_log=np.exp(forest_reg_predictions_test_log)
forest_reg_predictions_test_best_log=np.exp(forest_reg_prediction_test_best_log)

#df=pd.DataFrame({'datetime':df_test_datetime, 'count':forest_reg_predictions_test})
#df_best=pd.DataFrame({'datetime':df_test_datetime, 'count':forest_reg_predictions_test_best})
df_log=pd.DataFrame({'datetime':df_test_datetime, 'count':forest_reg_predictions_test_log})
df_best_log = pd.DataFrame({'datetime':df_test_datetime, 'count':forest_reg_predictions_test_best_log})

#df.to_csv('Forest file.csv', sep=',', encoding = 'utf-8', index=False)
#df_best.to_csv('Forest best file.csv', sep=',', encoding = 'utf-8', index=False)
df_log.to_csv('Forest log file.csv', sep=',', encoding = 'utf-8', index=False)
df_best_log.to_csv('Forest best log file.csv', sep=',', encoding = 'utf-8', index=False)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

Gb_reg = GradientBoostingRegressor()
Gb_reg.fit (X, Y)
Gb_reg_predictions = Gb_reg.predict(X2)
gb_df = pd.DataFrame({'datetime':df_test_datetime, 'count':Gb_reg_predictions})
gb_df.to_csv('Gb file.csv', sep=',', encoding = 'utf-8', index=False)

In [ ]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor()
xgb_reg.fit(X, Y)
xgb_reg_predictions = xgb_reg.predict(X2)
xgb_df = pd.DataFrame({'datetime':df_test_datetime, 'count':xgb_reg_predictions})
xgb_df.to_csv('Xgb file.csv', sep=',', encoding = 'utf-8', index=False)
